In [1]:
import numpy as np
import glob
#import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277
from tensorflow.keras.models import load_model


2024-03-21 16:41:47.339384: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 16:41:47.625907: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 16:41:47.625943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 16:41:47.671746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 16:41:47.771658: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2327685752529930539
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7345625467344083615
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-21 16:41:51.038990: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:41:51.191711: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:41:51.191743: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:41:51.306142: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 16:41:51.306184: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values
    
print(train_values.shape)
print(test_values.shape)

X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)
y_test1=to_categorical(y_test)

X = np.concatenate((X_train1, X_test1), axis = 0)
Y = np.concatenate((y_train1, y_test1), axis = 0)


(38751, 277)
(72794, 277)


In [4]:
def focal_loss(gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        loss = -K.pow(1 - pt, gamma) * K.log(pt)
        return loss

    return focal_loss_fixed

In [8]:

model_files = ['./models/mlpmodel.h5', './models/cnnmodel.h5', './models/lstmmodel.h5']
model_names = ['MLP','CNN', 'LSTM']

for model_file, model_name in zip(model_files, model_names):
        if model_name == 'LSTM':
                model = load_model(model_file, custom_objects={'focal_loss_fixed': focal_loss})
        else:
                model = load_model(model_file)

        feature_importance_df = pd.DataFrame()

        for class_num in range(1, 7):

                N_test = test_values[test_values[:,-2]==class_num]

                Xs_test = N_test[:,:-2]
                ys_test = N_test[:len(N_test),-2]

                Xs_test1 = Xs_test.reshape(-1, X_test.shape[1], 1)
                ys_test1 = to_categorical(ys_test)


                M = np.zeros((Xs_test1.shape[0], 23))

                        
                pred_k = model.predict(Xs_test1)     

                mean_importance_list = []
                for _ in range(20):
                    for slice_start in range(0, 275, 12):
                        x_permuted = np.copy(Xs_test1)
                        x_slice = Xs_test1[:, slice_start:slice_start+12]
                        x_slice_permuted = np.random.permutation(x_slice)
                        x_permuted[:, slice_start:slice_start + 12] = x_slice_permuted
                        pred_perm = model.predict(x_permuted)

                        importance = ((np.argmax(ys_test1, axis=1) - np.argmax(pred_perm, axis=1))**2 
                                      - (np.argmax(ys_test1, axis=1) - np.argmax(pred_k, axis=1))**2)
                        M[:, slice_start // 12] = importance

                    mean_importance = np.mean(M, axis=0)
                    mean_importance_list.append(mean_importance)

                mean_importance_mean = np.mean(mean_importance_list, axis=0)
                
                mean_importance = mean_importance_mean

                indices_sort = np.argsort(-1 * mean_importance)
                slices = np.arange(1, 24)
                feature_importance_normalized = (mean_importance - mean_importance.min()) / (mean_importance.max() - mean_importance.min())

                fig, ax = plt.subplots(1, 2, figsize = (15, 4))
                sns.barplot(x=np.arange(23), y=feature_importance_normalized[indices_sort], ax=ax[0],palette="pastel")
                ax[0].set_title('Feature importance per segment for the '+ str(model_name) + ' model for class '+ str(class_num)+ 'of ECGs')
                ax[0].set_xticks(np.arange(23))
                ax[0].set_xticklabels(slices[indices_sort].astype(int))
                ax[0].set_xlabel('Slice')
                ax[0].set_ylabel('Feature importance')

                ecg_normalized = (Xs_test[20, :] - Xs_test[20, :].min()) / (Xs_test[20, :].max() - Xs_test[20, :].min())
                ax[1].plot(np.arange(len(ecg_normalized)), ecg_normalized, label='ECG data')
                ax[1].plot(np.repeat(feature_importance_normalized, 12), label = 'Feature importance')
                ax[1].set_title('Feature importance per segment \nfor the '+ str(model_name) + ' model for class'+ str(class_num)+ 'of ECGs')
                ax[1].set_xlabel('Time')
                ax[1].set_ylabel('ECG signal / feature importance')
                ax[1].legend()
                image_name = 'output_pics/'+ str(model_name)+'_permutation_feature_importance_results for class' + str(class_num) + '.jpg'
                fig.savefig(image_name, dpi = 400)


                # Append the importance_dict to the feature_importance_df DataFrame
                feature_importance_df['class_' + str(class_num)] = feature_importance_normalized
        # Save the feature_importance_df DataFrame to a CSV file
        feature_importance_df.to_csv('./output_csv/pfi_'+ str(model_name) + '.csv', index=False)
                

 357/1843 [====>.........................] - ETA: 2s

KeyboardInterrupt: 